In [35]:
import pandas as pd
import numpy as np 
import pickle

from collections import Counter

# Data/Model Loading

In [123]:
SAMPLES_PARQUET = '../data/features/featureswithouttags.parquet'

samples = pd.read_parquet(SAMPLES_PARQUET)

feature_groups = [
    "audio_features",
    "metadata",
    "ranks",
    "social_features",
    "temporal_features",
    "album",
    "artist",
    "set",
    "track",
]

all_features = {f"{_group}" : [] for _group in feature_groups}


for feat_comb in samples.columns:
    _group = feat_comb[0]
    #print(feat_comb)
    _feature = feat_comb[1]
    all_features[_group].append(_feature)

all_features
#samples.head()

audio_features = samples['audio_features']
track_metadata = samples['metadata']
social_features = samples['social_features']
social_features.head()

,artist_discovery,artist_familiarity,artist_hotttnesss,song_currency,song_hotttnesss
track_id,,,,,
2,0.388990,0.386740,0.406370,0.000000,0.000000
3,0.388990,0.386740,0.406370,0.000000,0.000000
5,0.388990,0.386740,0.406370,0.000000,0.000000
10,0.557339,0.614272,0.798387,0.005158,0.354516
134,0.388990,0.386740,0.406370,0.000000,0.000000


In [37]:
MODEL_PKL = '../modeling/training/xgboost.pkl'
#loaded_model = pickle.load(open(MODEL_PKL))

model = None
with open(MODEL_PKL,'rb') as f:
    model = pickle.load(f)

# Playlist Creation
---


In [131]:
def create_playlist(tracks,social_features_list=None,random=False):
    if social_features_list is None:
        social_features_list = ['artist_discovery',
            'artist_familiarity',
            'artist_hotttnesss',
            'song_currency',
            'song_hotttnesss'
        ]
    social_features_tuple = [('social_features',_feat) for _feat in social_features_list]
    tracks = tracks.sort_values(by=social_features_tuple,ascending=False)
    return tracks[[('track','title'),('metadata','artist_name'),('track','genre_top'),('cluster','cluster')]]

## XGBoost Model

In [38]:
import xgboost as xgb
dtest_reg = xgb.DMatrix(audio_features)

In [97]:
y_test_pred = model.predict(dtest_reg)
y_test_pred

print(Counter(y_test_pred))

Counter({2.0: 9518, 1.0: 2610, 0.0: 1001})


In [121]:
clustered_samples = samples.copy()
clustered_samples[('cluster','cluster')] = y_test_pred
clustered_samples.head()

audio_features                                                    \
           acousticness danceability    energy instrumentalness  liveness   
track_id                                                                    
2              0.416675     0.675894  0.634476         0.010628  0.177647   
3              0.374408     0.528643  0.817461         0.001851  0.105880   
5              0.043567     0.745566  0.701470         0.000697  0.373143   
10             0.951670     0.658179  0.924525         0.965427  0.115474   
134            0.452217     0.513238  0.560410         0.019443  0.096567   

                                           metadata                     ...  \
         speechiness    tempo   valence  album_date         album_name  ...   
track_id                                                                ...   
2           0.159310  165.922  0.576661        None               None  ...   
3           0.461818  126.957  0.269240        None               None  ...   
5           0.124595  100.260  0.621661        None               None  ...   
10          0.032985  111.562  0.963590  2008-03-11  Constant Hitmaker  ...   
134         0.525519  114.290  0.894072        None               None  ...   

            track                \
         interest language_code   
track_id                          
2            4656            en   
3            1470            en   
5            1933            en   
10          54881            en   
134          1126            en   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
2         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293     None   
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...     514     None   
5         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1151     None   
10        Attribution-NonCommercial-NoDerivatives (aka M...   50135     None   
134       Attribution-NonCommercial-ShareAlike 3.0 Inter...     943     None   

                                             cluster  
         number publisher tags         title cluster  
track_id                                              
2             3      None   []          Food     2.0  
3             4      None   []  Electric Ave     2.0  
5             6      None   []    This World     2.0  
10            1      None   []       Freeway     2.0  
134           5      None   []  Street Music     2.0  

[5 rows x 302 columns]

In [184]:
tracks_for_physical = clustered_samples[clustered_samples['cluster','cluster']==2.0]
#tracks_for_physical_sorted[('track','genre_top')].value_counts()

## 	artist_discovery	artist_familiarity	artist_hotttnesss	song_currency	song_hotttnesss
## select one or more in order of likeness
create_playlist(tracks=tracks_for_physical,social_features_list=['artist_familiarity']).sample(100)
#tracks_for_physical_sorted[('track','genre_top')].value_counts()


,track,metadata,track,cluster
,title,artist_name,genre_top,cluster
track_id,,,,
39686,4 The Model feat. Camil - Damen,The Model,Electronic,2.0
16897,Meatje,D'r Sjaak,None,2.0
124528,Wait For You,Monolog,None,2.0
124765,Ninja,Indikings,None,2.0
21423,Birds Dazed And Confused,Terminal 11,Electronic,2.0
...,...,...,...,...
15297,Sticks to the Skin,1.6 Band,Rock,2.0
19250,Rag Doll Physics,Diablo Swing Orchestra,None,2.0


In [179]:
tracks_for_focus = clustered_samples[clustered_samples['cluster','cluster']==1.0]

## 	artist_discovery	artist_familiarity	artist_hotttnesss	song_currency	song_hotttnesss
## select one or more in order of likeness
print(tracks_for_focus[('track','genre_top')].value_counts())
create_playlist(tracks=tracks_for_focus,social_features_list=['artist_discovery']).sample()



(track, genre_top)
Rock                   589
Folk                   310
Electronic             250
Jazz                   224
Hip-Hop                212
Instrumental            72
Pop                     66
Old-Time / Historic     50
International           40
Blues                   29
Experimental             4
Name: count, dtype: int64


,track,metadata,track,cluster
,title,artist_name,genre_top,cluster
track_id,,,,
14585,The Dream of Sergei Prokofiev and Waltz,Ergo Phizmiz,Instrumental,1.0


In [161]:
tracks_for_relax = clustered_samples[clustered_samples['cluster','cluster']==0.0]

## 	artist_discovery	artist_familiarity	artist_hotttnesss	song_currency	song_hotttnesss
## select one or more in order of likeness
create_playlist(tracks=tracks_for_relax,social_features_list=['artist_familiarity'])
tracks_for_relax[('track','genre_top')].value_counts()

(track, genre_top)
Classical              265
Old-Time / Historic    221
Folk                   164
Rock                    60
Electronic              35
International           14
Pop                     12
Jazz                     4
Experimental             3
Blues                    2
Hip-Hop                  2
Name: count, dtype: int64